In [ ]:
import pandas as pd
import nltk
nltk.download("stopwords")

In [ ]:
with open('train.dat','r') as f:
    df = pd.DataFrame(l.split("\t") for l in f) 

In [ ]:
df.columns = ["rating","review"]
df["review"] = df["review"].str.lower().str.split()

In [ ]:
#Removal of stopwords and other unwanted things (only Alphabates present)
from nltk.corpus import stopwords
import re
stop = stopwords.words('english')
df["review"] = df["review"].apply(lambda x: [item for item in x if item not in stop])
df["review"] = df["review"].apply(lambda sentence :[word.replace(",", "").replace(".", "").replace("(","").replace(")","").replace("/","") for word in sentence])   
df["review"] = df["review"].apply(lambda line: [re.sub("[^a-zA-Z]+", "", word) for word in line])

In [ ]:
poslines = df.loc[df['rating'].isin(['+1'])]
neglines = df.loc[df['rating'].isin(['-1'])]

In [ ]:
#Determine frequency distribution from positive reviews and negative reviews. Only taking into consideration long words
wd_freq={}
trainfeatures = []
for pdf in poslines["review"]:
    poswords =[]
    for word in list(pdf):
        if(len(word)>=3 and str.isalpha(word)):
            wd_freq[word]= wd_freq.get(word, 0) + 1
            poswords.append(word)
    trainfeatures.append((poswords, '+1'))  
    
for pdf in neglines["review"]: 
    negwords =[]
    for word in list(pdf):
        if(len(word)>=3 and str.isalpha(word)):
            wd_freq[word]= wd_freq.get(word, 0) + 1
            negwords.append(word)
    trainfeatures.append((negwords, '-1')) 
print(len(trainfeatures))    

In [ ]:
#Preprocess test data
with open('test.dat','r') as f:
        test_df = pd.DataFrame(l.split("\t") for l in f)
        test_df.columns = ["review"]
        test_df["review"] = test_df["review"].str.lower().str.split()
        stop = stopwords.words('english')
        test_df["review"]= test_df["review"].apply(lambda x: [item for item in x if item not in stop])
        test_df["review"]=  test_df["review"].apply(lambda x :[word.replace(",", "").replace(".", "").replace("(","").replace(")","").replace("/","") for word in x])  
        test_df["review"]= test_df["review"].apply(lambda x: [re.sub("[^a-zA-Z]+", "", word) for word in x])           

In [ ]:
def k_neighbour_find(test_df,trainfeatures,wd_freq,length,k):
    import warnings
    from math import log
    count =0
    if(len(trainfeatures)<=k):
        warnings.warn('K is set to a value, that is less than input length')
    output = open("prediciton.dat", 'w')   
    for sentence in test_df["review"]:
        results=[]
        commonwords=[]
        for trainwords, trainlabel in trainfeatures:
            commonwords= [x for x in trainwords if x in sentence and len(x)>=3]
            match = 0.0
            for word in commonwords:
                 match = match + log(length/wd_freq[word])    
            results.append((match, trainlabel))    
        results.sort(reverse=True)
        top_k= [x[1] for x in results[:k]]
        positives= top_k.count('+1')
        negatives= top_k.count('-1')
        if negatives> positives:
            count = count + 1
            output.write("-1\n")
            print(count)
        else: 
            count = count + 1
            output.write("+1\n")
            print(count)
    output.close() 
    print("Job Done!!!")

In [ ]:
import math
length = len(poslines) + len(neglines)
x = math.sqrt(length)
k = math.floor(x) 
k = k * 2
print("K will "+ str(k) )  
k_neighbour_find(test_df,trainfeatures,wd_freq,length,k)